In [1]:
import plotly.express as px
import pandas as pd

data_cc = pd.read_csv('../data/cc_data.csv')

fig_histogram = px.histogram(data_cc, x='location',
                             title='Credit Card: Number of Transactions by Location',
                             labels={'location': 'Location'},
                             category_orders={'location': data_cc['location'].value_counts().index.tolist()}) 

fig_histogram.update_layout(bargap=0.2)
fig_histogram.show()


In [3]:
data_lc = pd.read_csv('../data/loyalty_data.csv')

fig_histogram = px.histogram(data_lc, x='location',
                             title='Loyalty Card: Number of Transactions by Location',
                             labels={'location': 'Location'},
                             category_orders={'location': data_lc['location'].value_counts().index.tolist()}) 

fig_histogram.update_layout(bargap=0.2)
fig_histogram.show()

In [4]:
fig_histogram = px.histogram(data_lc, x='location',
                             title='Loyalty Card and Credit Card: Number of Transactions by Location',
                             labels={'location': 'Location'},
                             category_orders={'location': data_lc['location'].value_counts().index.tolist()}) 

fig_histogram.update_layout(bargap=0.2)
fig_histogram.show()

# CC transactions per hour 

In [10]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load data
df = data_cc

# Convert the timestamp column to datetime type and extract the hour
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m/%d/%Y %H:%M')
df['hour'] = df['timestamp'].dt.hour

# Initialize Dash application
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Credit Card Transactions per Hour"),
    html.Button('Select All Locations', id='select-all-button'),
    dcc.Dropdown(
        id='location-dropdown',
        options=[{'label': loc, 'value': loc} for loc in df['location'].unique()],
        value=[df['location'].unique()[0]], 
        multi=True
    ),
    dcc.Graph(id='transaction-graph')
])

@app.callback(
    Output('location-dropdown', 'value'),
    [Input('select-all-button', 'n_clicks')],
    [dash.dependencies.State('location-dropdown', 'options')]
)
def select_all_locations(n_clicks, options):
    if n_clicks:
        return [option['value'] for option in options]
    raise dash.exceptions.PreventUpdate

@app.callback(
    Output('transaction-graph', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_graph(selected_locations):
    if not selected_locations:
        return px.bar()  
    filtered_df = df[df['location'].isin(selected_locations)]
    grouped = filtered_df.groupby(['location', 'hour']).size().reset_index(name='count')
    fig = px.bar(grouped, x='hour', y='count', color='location', barmode='group', title='Number of Transactions per Hour (Credit Card)')
    
    fig.update_xaxes(range=[0, 24], dtick=1)  # X axis from 0-24
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)


In [11]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load data
df = pd.read_csv('../data/loyalty-cc.csv')

# Convert the timestamp column to datetime type and extract the hour
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m/%d/%Y %H:%M')
df['hour'] = df['timestamp'].dt.date

# Initialize Dash application
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Loyalty Card Transactions per Date"),
    html.Button('Select All Locations', id='select-all-button'),
    dcc.Dropdown(
        id='location-dropdown',
        options=[{'label': loc, 'value': loc} for loc in df['location'].unique()],
        value=[df['location'].unique()[0]], 
        multi=True
    ),
    dcc.Graph(id='transaction-graph')
])

@app.callback(
    Output('location-dropdown', 'value'),
    [Input('select-all-button', 'n_clicks')],
    [dash.dependencies.State('location-dropdown', 'options')]
)
def select_all_locations(n_clicks, options):
    if n_clicks:
        return [option['value'] for option in options]
    raise dash.exceptions.PreventUpdate

@app.callback(
    Output('transaction-graph', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_graph(selected_locations):
    if not selected_locations:
        return px.bar()  
    filtered_df = df[df['location'].isin(selected_locations)]
    grouped = filtered_df.groupby(['location', 'hour']).size().reset_index(name='count')
    fig = px.bar(grouped, x='hour', y='count', color='location', barmode='group', title='Number of Transactions per Date (Loyalty Card)')
    fig.update_layout(xaxis_title='Date')
    # fig.update_xaxes(range=[0, 24], dtick=1)  # X axis from 0-24
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8049)


In [20]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
from dash.exceptions import PreventUpdate

# Load data
cc_df = data_lc
cc_df['timestamp'] = pd.to_datetime(cc_df['timestamp'], format='%m/%d/%Y %H:%M')
cc_df['date'] = cc_df['timestamp'].dt.date  # Keep the full date

loyalty_df = data_lc
loyalty_df['timestamp'] = pd.to_datetime(loyalty_df['timestamp'], format='%m/%d/%Y')
loyalty_df['date'] = loyalty_df['timestamp'].dt.date  # Keep the full date

# Initialize Dash app
app = dash.Dash(__name__)

# Set up app layout
app.layout = html.Div([
    html.H1("Comparison between c   redit card and loyalty card"),
    html.Button('Select All Locations', id='select-all-button'),
    dcc.Dropdown(
        id='location-dropdown',
        options=[{'label': loc, 'value': loc} for loc in pd.concat([cc_df['location'], loyalty_df['location']]).unique()],
        multi=True
    ),
    dcc.RadioItems(
        id='time-range-selector',
        options=[
            {'label': '', 'value': 'week'},
            {'label': '', 'value': 'day'}
        ],
        value='week',
        labelStyle={'display': 'inline-block'}
    ),
    dcc.DatePickerSingle(
        id='date-picker-single',
        min_date_allowed=cc_df['date'].min(),
        max_date_allowed=cc_df['date'].max(),
        initial_visible_month=cc_df['date'].min()
    ),
    dcc.Graph(id='transaction-graph'),
    dcc.Store(id='selected-bar-info'),
    html.Div(id='transaction-table')
])

# Callback to select all locations
@app.callback(
    Output('location-dropdown', 'value'),
    [Input('select-all-button', 'n_clicks')],
    [dash.dependencies.State('location-dropdown', 'options')]
)
def select_all_locations(n_clicks, options):
    if n_clicks:
        return [option['value'] for option in options]
    raise dash.exceptions.PreventUpdate

# Callback to update the graph
@app.callback(
    Output('transaction-graph', 'figure'),
    [Input('location-dropdown', 'value'),
     Input('time-range-selector', 'value'),
     Input('date-picker-single', 'date')]
)
def update_graph(selected_locations, time_range, selected_date):
    if not selected_locations:
        raise PreventUpdate
    
    filtered_cc = cc_df[cc_df['location'].isin(selected_locations)]
    filtered_loyalty = loyalty_df[loyalty_df['location'].isin(selected_locations)]

    if time_range == 'day':
        if not selected_date:
            raise PreventUpdate
        filtered_cc = filtered_cc[filtered_cc['date'] == pd.to_datetime(selected_date)]
        filtered_loyalty = filtered_loyalty[filtered_loyalty['date'] == pd.to_datetime(selected_date)]
    
    # Count transactions per location per day
    cc_counts = filtered_cc.groupby(['date', 'location']).size().reset_index(name='cc_count')
    loyalty_counts = filtered_loyalty.groupby(['date', 'location']).size().reset_index(name='loyalty_count')

    # Merge data
    merged_data = pd.merge(cc_counts, loyalty_counts, on=['date', 'location'], how='outer').fillna(0)
    
    # Plot
    fig = px.bar(merged_data, x='date', y=['cc_count', 'loyalty_count'], barmode='group', color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.update_traces(customdata=merged_data['location'].values)

    return fig

# Callback to toggle date picker visibility
@app.callback(
    Output('date-picker-single', 'style'),
    [Input('time-range-selector', 'value')],
)
def toggle_date_picker_visibility(selector_value):
    if selector_value == 'day':
        return {'display': 'block'}
    return {'display': 'none'}

# Callback to store click data
@app.callback(
    Output('selected-bar-info', 'data'),
    [Input('transaction-graph', 'clickData')]
)
def store_click_data(clickData):
    if clickData:
        point = clickData['points'][0]
        date = point['x']
        location = point['customdata']
        return {'date': date, 'location': location}
    raise PreventUpdate

# Callback to display transactions
@app.callback(
    Output('transaction-table', 'children'),
    [Input('selected-bar-info', 'data')]
)
def display_transactions(selected_bar_info):
    if not selected_bar_info:
        return "Click one of the columns to show details."

    date = pd.to_datetime(selected_bar_info['date']).date()
    location = selected_bar_info['location']

    cc_transactions = cc_df[(cc_df['date'] == date) & (cc_df['location'] == location)]
    loyalty_transactions = loyalty_df[(loyalty_df['date'] == date) & (loyalty_df['location'] == location)]

    cc_transactions_table = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in cc_transactions.columns])),
        html.Tbody([
            html.Tr([html.Td(cc_transactions.iloc[i][col]) for col in cc_transactions.columns])
            for i in range(len(cc_transactions))
        ])
    ])

    loyalty_transactions_table = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in loyalty_transactions.columns])),
        html.Tbody([
            html.Tr([html.Td(loyalty_transactions.iloc[i][col]) for col in loyalty_transactions.columns])
            for i in range(len(loyalty_transactions))
        ])
    ])

    return html.Div([
        html.H3(f"{location} - {date} credit_card transactions"),
        cc_transactions_table,
        html.H3(f"{location} - {date} loyalty_card transactions"),
        loyalty_transactions_table
    ])

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
